You just got hired as the first and only data practitioner at a small business experiencing exponential growth. The company needs more structured processes, guidelines, and standards. Your first mission is to structure the human resources data. The data is currently scattered across teams and files and comes in various formats: Excel files, CSVs, JSON files...

You'll work with the following data in the `datasets` folder:
- __Office addresses__
    - Saved in `office_addresses.csv`. 
    - If the value for office is `NaN`, then the employee is remote.
- __Employee addresses__
    - Saved on the first tab of `employee_information.xlsx`.
- __Employee emergency contacts__ 
    - Saved on the second tab of `employee_information.xlsx`; this tab is called `emergency_contacts`. 
    - However, this sheet was edited at some point, and ***the headers were removed***! The HR manager let you know that they should be: `employee_id`, `last_name`, `first_name`, `emergency_contact`, `emergency_contact_number`, and `relationship`.
- __Employee roles, teams, and salaries__ 
    - This information has been exported from the company's human resources management system into a JSON file titled `employee_roles.json`. Here are the first few lines of that file:
```
{"A2R5H9":
  {
    "title": "CEO",
    "monthly_salary": "$4500",
    "team": "Leadership"
  },
 ...
}
```

In [1]:
import pandas as pd
# Start coding here... 

In [2]:
ee_info = pd.read_excel("datasets/employee_information.xlsx")
ee_contact = pd.read_excel("datasets/employee_information.xlsx", sheet_name="emergency_contacts",names=["employee_id","last_name","first_name","emergency_contact","emergency_contact_number","relationship"],header=None)
ee_roles = pd.read_json("datasets/employee_roles.json",orient="index")
office_address = pd.read_csv("datasets/office_addresses.csv")

In [3]:
ee_contact

,employee_id,last_name,first_name,emergency_contact,emergency_contact_number,relationship
0,A2R5H9,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother
1,H8K0L6,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister
2,G4R7V0,Sagal,Gemma,John Newmark,+1-202-555-194,Husband
3,M1Z7U9,Coates,Tig,Venus Noone,+1-202-555-0130,Wife


In [4]:
ee_roles.reset_index(inplace=True)
ee_roles.rename(columns={'index':'employee_id'}, inplace=True)
ee_roles

,employee_id,title,monthly_salary,team
0,A2R5H9,CEO,$4500,Leadership
1,H8K0L6,CFO,$4500,Leadership
2,G4R7V0,Business Developer,$3000,Sales
3,M1Z7U9,Office Manager,$2000,People Operations


In [5]:
# unified dataframe
info_roles = pd.merge(ee_info,ee_roles,on="employee_id",how="outer")
info_roles_contact = pd.merge(info_roles,ee_contact,on="employee_id")
employees_final = pd.merge(info_roles_contact,office_address,left_on="employee_country",right_on="office_country",how="outer")

In [6]:
employees_final

,employee_id,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number,title,monthly_salary,team,last_name,first_name,emergency_contact,emergency_contact_number,relationship,office,office_country,office_city,office_street,office_street_number
0,A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9,CEO,$4500,Leadership,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother,Leuven Office,BE,Leuven,Martelarenlaan,38.0
1,H8K0L6,Siff,Tara,GB,London,Baker Street,221,CFO,$4500,Leadership,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister,WeWork Office,GB,London,Old Street,207.0
2,G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66,Business Developer,$3000,Sales,Sagal,Gemma,John Newmark,+1-202-555-194,Husband,ESB Office,US,New York City,Fifth Avenue,350.0
3,M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7,Office Manager,$2000,People Operations,Coates,Tig,Venus Noone,+1-202-555-0130,Wife,NaN,NaN,NaN,NaN,NaN


In [7]:
ee_contact

,employee_id,last_name,first_name,emergency_contact,emergency_contact_number,relationship
0,A2R5H9,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother
1,H8K0L6,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister
2,G4R7V0,Sagal,Gemma,John Newmark,+1-202-555-194,Husband
3,M1Z7U9,Coates,Tig,Venus Noone,+1-202-555-0130,Wife


In [8]:
employees_final.fillna("Remote", inplace=True)
employees_final.drop(columns=["last_name","first_name"],axis=1,inplace=True)
#employees_final.loc["1","H8K0L6"] = "H8KOL6"
employees_final

,employee_id,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number,title,monthly_salary,team,emergency_contact,emergency_contact_number,relationship,office,office_country,office_city,office_street,office_street_number
0,A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9,CEO,$4500,Leadership,Opie Hurst,+32-456-5556-84,Brother,Leuven Office,BE,Leuven,Martelarenlaan,38.0
1,H8K0L6,Siff,Tara,GB,London,Baker Street,221,CFO,$4500,Leadership,Wendy de Matteo,+44-020-5554-333,Sister,WeWork Office,GB,London,Old Street,207.0
2,G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66,Business Developer,$3000,Sales,John Newmark,+1-202-555-194,Husband,ESB Office,US,New York City,Fifth Avenue,350.0
3,M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7,Office Manager,$2000,People Operations,Venus Noone,+1-202-555-0130,Wife,Remote,Remote,Remote,Remote,Remote


In [9]:
employees_final.set_index("employee_id",inplace=True)

In [10]:
#employees_final.loc["1","H8K0L6"] = "H8KOL6"
#employees_final.drop(index="1",inplace=True)

In [11]:
employees_final

,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number,title,monthly_salary,team,emergency_contact,emergency_contact_number,relationship,office,office_country,office_city,office_street,office_street_number
employee_id,,,,,,,,,,,,,,,,,
A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9,CEO,$4500,Leadership,Opie Hurst,+32-456-5556-84,Brother,Leuven Office,BE,Leuven,Martelarenlaan,38.0
H8K0L6,Siff,Tara,GB,London,Baker Street,221,CFO,$4500,Leadership,Wendy de Matteo,+44-020-5554-333,Sister,WeWork Office,GB,London,Old Street,207.0
G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66,Business Developer,$3000,Sales,John Newmark,+1-202-555-194,Husband,ESB Office,US,New York City,Fifth Avenue,350.0
M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7,Office Manager,$2000,People Operations,Venus Noone,+1-202-555-0130,Wife,Remote,Remote,Remote,Remote,Remote


In [12]:
columns_names = ["employee_first_name", "employee_last_name", "employee_country", 
                 "employee_city", "employee_street", "employee_street_number", 
                 "emergency_contact", "emergency_contact_number", "relationship", 
                 "monthly_salary", "team", "title", "office", "office_country", 
                 "office_city", "office_street", "office_street_number"]

In [13]:
employees_final = employees_final[columns_names]
employees_final

,employee_first_name,employee_last_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,monthly_salary,team,title,office,office_country,office_city,office_street,office_street_number
employee_id,,,,,,,,,,,,,,,,,
A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,$4500,Leadership,CEO,Leuven Office,BE,Leuven,Martelarenlaan,38.0
H8K0L6,Tara,Siff,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,$4500,Leadership,CFO,WeWork Office,GB,London,Old Street,207.0
G4R7V0,Gemma,Sagal,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband,$3000,Sales,Business Developer,ESB Office,US,New York City,Fifth Avenue,350.0
M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,$2000,People Operations,Office Manager,Remote,Remote,Remote,Remote,Remote
